### trying to impliment the espcn paper (paper number 3)

## Methodology:
1. convoloving the filter as usual on the images with relu activation. (test with tanh)
2. before passing it to ANNs we apply a sub-pixel layer 
3. pass the network in the training loop 

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#laoding the data.



In [ ]:
class CNN_diff_multiscale(nn.Module):
    def __init__(self, scale_factor=2):
        super(CNN_diff_multiscale, self).__init__()
        self.scale_factor = scale_factor

        # Main branch
        self.conv1 = nn.Conv2d(1, 64, kernel_size=9, padding=9//2)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=3, padding=3//2)
        self.conv3 = nn.Conv2d(32, 16, kernel_size=3, padding=3//2)

        # Parallel branch with larger receptive field
        self.conv_large = nn.Conv2d(1, 32, kernel_size=13, padding=13//2)

        # Upsampling layers
        self.upsample = nn.PixelShuffle(self.scale_factor)
        
        # Final convolution
        self.conv_final = nn.Conv2d(16 + 32 // (self.scale_factor ** 2), 1, kernel_size=5, padding=5//2)

    def forward(self, x):
        # Main branch
        main = F.relu(self.conv1(x))
        main = F.relu(self.conv2(main))
        main = F.relu(self.conv3(main))

        # Parallel branch
        large = F.relu(self.conv_large(x))

        # Upsample both branches
        main_up = self.upsample(main)
        large_up = self.upsample(large)

        # Concatenate upsampled features
        combined = torch.cat([main_up, large_up], dim=1)

        # Final convolution
        output = self.conv_final(combined)

        return output

    def multi_scale_loss(pred, target, scale_weights=[1.0, 0.5, 0.25]):
        losses = []
        for i, weight in enumerate(scale_weights):
            if i > 0:
                pred = F.interpolate(pred, scale_factor=0.5, mode='bilinear', align_corners=False)
                target = F.interpolate(target, scale_factor=0.5, mode='bilinear', align_corners=False)
            losses.append(weight * F.mse_loss(pred, target))
        return sum(losses)
    def random_scale_augmentation(image, min_scale=0.5, max_scale=2.0):
        scale = random.uniform(min_scale, max_scale)
        return F.interpolate(image, scale_factor=scale, mode='bilinear', align_corners=False)

# During training
augmented_input = random_scale_augmentation(original_input)
output = model(augmented_input)
loss = multi_scale_loss(output, original_input)